<a href="https://colab.research.google.com/github/escofresco/makeschool_ds22_keras_for_mnist/blob/main/Keras_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN with Keras

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np

# get train and test data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(y_train[0])
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)
y_train = to_categorical(y_train) # one hot encode
y_test = to_categorical(y_test) # one hot encode

y_train[0]

11493376/11490434 [==============================] - 0s 0us/step
5


array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [8]:
y_train.shape

(60000, 10)

In [9]:
#set up the model
model = Sequential()

#add convultional layer
model.add(Conv2D(32, (1, 1), padding="same", activation="relu"))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))

model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

#add some other Dense layers here and Flatten()
model.add(Flatten())

model.add(Dense(10, activation='softmax'))

#compile the model by setting the optimizer and loss and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#fit to train and test
model.fit(X_train.astype(np.float32), y_train.astype(np.float32), epochs=3)

#prediction
model.predict(X_test)

Epoch 1/3
1875/1875 [==============================] - 10s 6ms/step - loss: 0.1739 - accuracy: 0.9639
Epoch 2/3
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0517 - accuracy: 0.9840
Epoch 3/3
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0353 - accuracy: 0.9886


array([[1.0866094e-09, 1.2687811e-10, 7.7456100e-08, ..., 9.9999404e-01,
        1.2006541e-08, 4.8774325e-08],
       [1.2833297e-07, 3.3777005e-07, 9.9993074e-01, ..., 2.0781315e-12,
        3.0194610e-07, 1.8196137e-12],
       [1.1950154e-06, 9.9893111e-01, 1.5083702e-05, ..., 1.3510663e-04,
        6.5488572e-04, 7.7661235e-07],
       ...,
       [9.0429356e-15, 5.4395932e-11, 1.1140157e-14, ..., 2.4313994e-08,
        4.2886191e-07, 1.1981666e-09],
       [2.2037809e-09, 6.7036970e-15, 9.0594178e-11, ..., 2.8249003e-09,
        3.9374114e-05, 3.0145790e-09],
       [7.0242643e-09, 2.7085610e-12, 1.3136325e-08, ..., 5.5189651e-12,
        2.5590250e-08, 1.2793984e-10]], dtype=float32)

# CNN with NumPy

In [10]:
import skimage.data
import skimage.color
img = skimage.color.rgb2gray( skimage.data.chelsea() )

In [29]:
def conv(img, conv_filter):  
    if len(img.shape) > 2 or len(conv_filter.shape) > 3: 
      if img.shape[-1] != conv_filter.shape[-1]:  
              print("Error: Number of channels in both image and filter must match.")  
              sys.exit()  
      if conv_filter.shape[1] != conv_filter.shape[2]: 
          print('Error: Filter must be a square matrix. I.e. number of rows and columns must match.')  
          sys.exit()  
      if conv_filter.shape[1]%2==0: 
          print('Error: Filter must have an odd size. I.e. number of rows and columns must be odd.')  
          sys.exit()

    feature_maps = np.zeros((img.shape[0]-conv_filter.shape[1]+1,   
                                img.shape[1]-conv_filter.shape[1]+1,   
                                conv_filter.shape[0]))  

    for filter_num in range(conv_filter.shape[0]):  
        print("Filter ", filter_num + 1)  
        curr_filter = conv_filter[filter_num, :]
        if len(curr_filter.shape) > 2:  
            conv_map = conv_(img[:, :, 0], curr_filter[:, :, 0]) 
            for ch_num in range(1, curr_filter.shape[-1]):
                conv_map = conv_map + conv_(img[:, :, ch_num],   
                                  curr_filter[:, :, ch_num])  
        else: 
              conv_map = conv_(img, curr_filter)  
        feature_maps[:, :, filter_num] = conv_map
    return feature_maps

In [26]:
def relu(feature_map):  
   #Preparing the output of the ReLU activation function.  
   relu_out = np.zeros(feature_map.shape)  
   for map_num in range(feature_map.shape[-1]):  
       for r in np.arange(0,feature_map.shape[0]):  
           for c in np.arange(0, feature_map.shape[1]):  
               relu_out[r, c, map_num] = np.max(feature_map[r, c, map_num], 0)

In [25]:
def pooling(feature_map, size=2, stride=2):  
    pool_out = np.zeros((np.uint16((feature_map.shape[0]-size+1)/stride),  
                            np.uint16((feature_map.shape[1]-size+1)/stride),  
                            feature_map.shape[-1]))  
    for map_num in range(feature_map.shape[-1]):  
        r2 = 0  
        for r in np.arange(0,feature_map.shape[0]-size-1, stride):  
            c2 = 0  
            for c in np.arange(0, feature_map.shape[1]-size-1, stride):  
                pool_out[r2, c2, map_num] = np.max(feature_map[r:r+size,  c:c+size])  
                c2 = c2 + 1  
                r2 = r2 +1  

In [33]:
l1_filter = np.zeros((2,3,3))
l1_filter[0, :, :] = np.array([[[-1, 0, 1],   
                                   [-1, 0, 1],   
                                   [-1, 0, 1]]])  
l1_filter[1, :, :] = np.array([[[1,   1,  1],   
                                   [0,   0,  0],   
                                   [-1, -1, -1]]])

In [32]:
## Layer 1
l1_feature_map = conv(img, l1_filter)
l1_feature_map_relu = relu(l1_feature_map)
l1_feature_map_relu_pool = pooling(l1_feature_map_relu, 2, 2)

## Layer 2
l2_filter = numpy.random.rand(3, 5, 5, l1_feature_map_relu_pool.shape[-1])  
l2_feature_map = conv(l1_feature_map_relu_pool, l2_filter)  
l2_feature_map_relu = relu(l2_feature_map)  
l2_feature_map_relu_pool = pooling(l2_feature_map_relu, 2, 2)  

## Layer 3
l3_filter = numpy.random.rand(1, 7, 7, l2_feature_map_relu_pool.shape[-1])  
l3_feature_map = conv(l2_feature_map_relu_pool, l3_filter)  
l3_feature_map_relu = relu(l3_feature_map)  
l3_feature_map_relu_pool = pooling(l3_feature_map_relu, 2, 2)

AttributeError: ignored